In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@eteproject.dfs.core.windows.net/products")

In [0]:
display(df)

In [0]:
df= df.drop("_rescued_data")

### functions

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_ete.bronze.discountFunc(discounted_price DOUBLE)
returns double 
LANGUAGE sql
return discounted_price*0.90

In [0]:
%sql
select product_id,price,databricks_ete.bronze.discountFunc(price) as discounted_price from products


In [0]:
df = df.withColumn("discounted_price", expr("databricks_ete.bronze.discountFunc(price)"))

create function using  python

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_ete.bronze.upperFunc(p_brand STRING )
returns STRING
LANGUAGE PYTHON AS
$$
return p_brand.upper()
$$

In [0]:
%sql 
SELECT product_id , brand ,databricks_ete.bronze.upperFunc(brand) from products

In [0]:


df.write.format("delta").mode("overwrite").save("abfss://silver@eteproject.dfs.core.windows.net/products")

In [0]:
%sql
create schema if not exists databricks_ete.silver

In [0]:
%sql 
create table if not exists databricks_ete.silver.products_silver
using delta
location "abfss://silver@eteproject.dfs.core.windows.net/products"
